In [105]:
import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
import h5py
import numpy as np

In [106]:
VALID_SPLIT = 0.2
TEST_SPLIT = 0.2

### Read data backout

In [107]:
def read_hdf(file_path):
    return_dict = []
    with h5py.File(file_path, 'r') as hf:
        print(list(hf.keys()))
        print(file_path[9:-5])
        dataset = hf[file_path[9:-5]]
        print(len(list(dataset.keys())))
        keys = list(dataset.keys())
        for i in keys:
            return_dict.append({
                "name":i.replace("-","/"),
                "data":dataset[i][:]
            })
        return return_dict

In [108]:
genre = ['Experimental', 'Electronic', 'Rock', 'Instrumental', 'Pop', 'Folk', 'Hip-Hop', 'International']

In [109]:
#!unzip mel_spec/update_feature.zip
# Move all files into mel_sepc then

In [21]:
hdf5_dict["Experimental"] = read_hdf("mel_spec/Experimental.hdf5")

['Experimental']
Experimental
999


In [135]:
mel_spec_base_path = "mel_spec/"
for i in genre:
    output = read_hdf(mel_spec_base_path+i+".hdf5") # list of dictionaries
    hdf5_dict[i] = output


In [48]:
hdf5_dict["International"][0]["data"]

(128, 646)

In [120]:
# label_reference = {}
# for index,value in enumerate(hdf5_dict.keys()):
#     label_reference[value] = index

In [1]:
label_reference = genres_dict = {
    'Electronic': 0, 
    'Experimental': 1, 
    'Folk': 2, 
    'Hip-Hop': 3, 
    'Instrumental': 4,
    'International': 5, 
    'Pop': 6, 
    'Rock': 7  
}

In [122]:
def splitting_test_valid(dictionary,test_size,valid_size):
    x = []
    y = []
    for genres,datas in dictionary.items():
        for i in datas:
            x.append(i['data'])
            y.append(label_reference[genres])
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = test_size)
    x_train,x_valid,y_train,y_valid = train_test_split(x_train,y_train,test_size = valid_size)
    return x_train,x_valid,x_test,y_train,y_valid,y_test

In [123]:
x_train,x_valid,x_test,y_train,y_valid,y_test = splitting_test_valid(hdf5_dict,0.2,0.2)

In [124]:

x_train = np.rollaxis(np.dstack(x_train),-1)
x_valid = np.rollaxis(np.dstack(x_valid),-1)
x_test = np.rollaxis(np.dstack(x_test),-1)
y_train = np.rollaxis(np.dstack(y_train),-1)
y_valid = np.rollaxis(np.dstack(y_valid),-1)
y_test = np.rollaxis(np.dstack(y_test),-1)


In [125]:
x_train = x_train[...,np.newaxis]
x_valid = x_valid[...,np.newaxis]
x_test = x_test[...,np.newaxis]

In [126]:
print(x_train.shape)
print(x_valid.shape)
print(x_test.shape)
print(y_train.shape)
print(y_valid.shape)
print(y_test.shape)


(5116, 128, 646, 1)
(1279, 128, 646, 1)
(1599, 128, 646, 1)
(5116, 1, 1)
(1279, 1, 1)
(1599, 1, 1)


### Model 

In [127]:
'''
Since the task is not overly ccomplex, choose to use LeNet 5
'''
from keras import layers
input_shape = (x_train.shape[1], x_train.shape[2], x_train.shape[3])
# inputs = keras.Input(shape=input_shape) 

def model(input_shape,label_reference):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=input_shape))
    model.add(tf.keras.layers.Conv2D(32, 3, strides=2, padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(128,activation = 'relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(len(label_reference), activation='softmax'))
    return model



In [128]:
model = model(input_shape,label_reference)

In [129]:
optimizer = keras.optimizers.SGD(learning_rate = 0.0005)
model.compile(optimizer = optimizer, loss = "SparseCategoricalCrossentropy",metrics = ["accuracy"])

### Fitting and training the data

In [133]:
model.fit(x_train,y_train,validation_data = (x_valid,y_valid),epochs=10,batch_size = 4)

Epoch 1/10
1279/1279 [==============================] - 158s 123ms/step - loss: 2.2853 - accuracy: 0.2353 - val_loss: 2.2745 - val_accuracy: 0.3229
Epoch 2/10
1279/1279 [==============================] - 164s 128ms/step - loss: 2.0047 - accuracy: 0.3223 - val_loss: 2.1041 - val_accuracy: 0.3182
Epoch 3/10
1279/1279 [==============================] - 161s 126ms/step - loss: 1.8213 - accuracy: 0.3853 - val_loss: 3.8077 - val_accuracy: 0.2737
Epoch 4/10
1279/1279 [==============================] - 188s 147ms/step - loss: 1.7310 - accuracy: 0.4097 - val_loss: 1.9523 - val_accuracy: 0.3995
Epoch 5/10
1279/1279 [==============================] - 190s 149ms/step - loss: 1.6480 - accuracy: 0.4328 - val_loss: 1.9483 - val_accuracy: 0.4011
Epoch 6/10
1279/1279 [==============================] - 171s 134ms/step - loss: 1.5350 - accuracy: 0.4734 - val_loss: 2.3703 - val_accuracy: 0.3831
Epoch 7/10
1279/1279 [==============================] - 162s 127ms/step - loss: 1.3678 - accuracy: 0.5323 - val_

### Checking the result

In [134]:
error,accuracy = model.evaluate(x_test[0:4],y_test[0:4],verbose = 1)
print(f"Error is {error} and accuracy is {accuracy}")

1/1 [==============================] - 0s 357ms/step - loss: 0.4062 - accuracy: 0.7500
Error is 0.40623772144317627 and accuracy is 0.75
